# Geoboundary Data Preparation

In [1]:
# Data Wrangling
import pandas as pd
import numpy as np

In [2]:
# import data
data = pd.read_csv("../data/CEWS_SSUC_DB_En_v1.0.csv", encoding = "ISO-8859-1")

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 562491 entries, 0 to 562490
Data columns (total 11 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   Start_date_of_CEWS_period  562491 non-null  object
 1   RegionCode                 562491 non-null  object
 2   RegionName                 562491 non-null  object
 3   RuralUrbanFlag             562491 non-null  object
 4   CMACAFlag                  562491 non-null  object
 5   IndustryCode               562491 non-null  object
 6   IndustryName               562491 non-null  object
 7   Number_business_locations  562491 non-null  object
 8   Subsidy_amount             562491 non-null  object
 9   Supported_employees        562491 non-null  object
 10  CEWS_rehire_count          562491 non-null  object
dtypes: object(11)
memory usage: 47.2+ MB


In [4]:
# replace the suppressed data as na
na_removed_data = data.replace('X', np.NaN)

In [5]:
# Filter at province level
data_province = na_removed_data[na_removed_data.RegionCode.str.len() == 2]
data_province.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9631 entries, 0 to 562011
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Start_date_of_CEWS_period  9631 non-null   object
 1   RegionCode                 9631 non-null   object
 2   RegionName                 9631 non-null   object
 3   RuralUrbanFlag             9631 non-null   object
 4   CMACAFlag                  9631 non-null   object
 5   IndustryCode               9631 non-null   object
 6   IndustryName               9631 non-null   object
 7   Number_business_locations  9631 non-null   object
 8   Subsidy_amount             6618 non-null   object
 9   Supported_employees        6837 non-null   object
 10  CEWS_rehire_count          9631 non-null   object
dtypes: object(11)
memory usage: 902.9+ KB


In [6]:
colnames = ["RegionCode", "RegionName", "RuralUrbanFlag", "CMACAFlag"]
for co in colnames:
    print(data_province[co].unique())

['10' '11' '12' '13' '24' '35' '46' '47' '48' '59' '60' '61' '62']
['Newfoundland and Labrador' 'Prince Edward Island' 'Nova Scotia'
 'New Brunswick' 'Quebec' 'Ontario' 'Manitoba' 'Saskatchewan' 'Alberta'
 'British Columbia' 'Yukon' 'Northwest Territories' 'Nunavut']
['Not applicable']
['Not applicable']


In [7]:
# Seems like RuralUrbanFlag and CMACAFlag does not play an import role in the province dataset, we can drop it.
data_province.drop(columns=["RuralUrbanFlag", "CMACAFlag"], inplace=True)
data_province.head()

C:\ProgramData\Miniconda3\lib\site-packages\pandas\core\frame.py:4167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,Start_date_of_CEWS_period,RegionCode,RegionName,IndustryCode,IndustryName,Number_business_locations,Subsidy_amount,Supported_employees,CEWS_rehire_count
0,2020-03-15,10,Newfoundland and Labrador,11,"Agriculture, forestry, fishing and hunting",30,"823,000",362,0
1,2020-03-15,10,Newfoundland and Labrador,111,Crop production,10,NaN,90,0
2,2020-03-15,10,Newfoundland and Labrador,112,Animal production and aquaculture,10,NaN,NaN,0
3,2020-03-15,10,Newfoundland and Labrador,113,Forestry and logging,5,NaN,NaN,0
4,2020-03-15,10,Newfoundland and Labrador,114,"Fishing, hunting and trapping",5,NaN,NaN,0


In [8]:
# Inorder to reuse the code:
def cews_getProvincialData(data, removeRuralUrbanFlag = True,  removeCMACAFlag = True):
    """
    This function will filter out data for all provinces
    
    input: A dataframe
    
    output: A dataframe
    """
    
    # Filter at province level
    data_province = data[data.RegionCode.str.len() == 2]
    
    # Seems like RuralUrbanFlag and CMACAFlag does not play an import role in the province dataset, we can drop it.
    if removeCMACAFlag:
        data_province.drop(columns=["CMACAFlag"], inplace=True)
    if removeRuralUrbanFlag:
        data_province.drop(columns=["RuralUrbanFlag"], inplace=True)
        
    return data_province
    

In [9]:
# cews_getProvincialData(na_removed_data)

In [10]:
# Now, lets investigate on the industry column, to make sure we don't add the same value multiple times.
data_province_totalIndustry = data_province[data_province.IndustryCode == "TOTAL"]
data_province_totalIndustry.drop(columns = ["IndustryCode", "IndustryName"], inplace=True)
data_province_totalIndustry.head(5)

C:\ProgramData\Miniconda3\lib\site-packages\pandas\core\frame.py:4167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,Start_date_of_CEWS_period,RegionCode,RegionName,Number_business_locations,Subsidy_amount,Supported_employees,CEWS_rehire_count
98,2020-03-15,10,Newfoundland and Labrador,"3,835","71,687,000","36,751",385
2648,2020-03-15,11,Prince Edward Island,"1,405","22,813,000","12,427",115
4006,2020-03-15,12,Nova Scotia,"7,100","134,082,000","70,253",715
6736,2020-03-15,13,New Brunswick,"5,865","105,072,000","52,785",540
10618,2020-03-15,24,Quebec,"59,980","1,396,578,000","678,340","8,335"


In [11]:
data_province_totalIndustry[data_province_totalIndustry.RegionCode == str(59)]

,Start_date_of_CEWS_period,RegionCode,RegionName,Number_business_locations,Subsidy_amount,Supported_employees,CEWS_rehire_count
59327,2020-03-15,59,British Columbia,"46,135","994,156,000","475,527","4,770"
135828,2020-04-12,59,British Columbia,"51,770","1,153,457,000","481,023","5,655"
213026,2020-05-10,59,British Columbia,"50,595","1,147,230,000","513,069","7,080"
285439,2020-06-07,59,British Columbia,"43,885","1,035,503,000","466,529","5,785"
357288,2020-07-05,59,British Columbia,"42,850","938,129,000","539,419","4,135"
427172,2020-08-02,59,British Columbia,"40,015","840,389,000","525,213","2,935"
493103,2020-08-30,59,British Columbia,"35,245","469,622,000","455,154","2,605"
552950,2020-09-27,59,British Columbia,"29,615","319,700,000","397,529","2,150"


In [12]:
# CEWS_rehire_count: Estimate of the total number of business locations having an indicator of retroactive rehire of one or more employees during the period.
# Number_business_location: Estimate of the total number of business locations claiming a Canadian Emergency Wage Subsidy.

# I am just wondering if we can develope a measurement: CEWS_rehire_count/Number_business_location, to figure out what proportion of those businesses that applies for the CEWS,
# rehire at least one new employee. 

# export the data
data_province_totalIndustry.to_csv("data_province_totalIndustry.csv")

In [13]:
# Let's investigate furthur. We just looked at the province level, now, let's move on to the next level(cities with CMA & CA flag).
# CMA flag: Census Metropolitan Areas
# CA flag: Census Agglomerations

In [14]:
data_CMA = na_removed_data[na_removed_data.IndustryCode == "TOTAL"]
data_CMA.head(5)

,Start_date_of_CEWS_period,RegionCode,RegionName,RuralUrbanFlag,CMACAFlag,IndustryCode,IndustryName,Number_business_locations,Subsidy_amount,Supported_employees,CEWS_rehire_count
98,2020-03-15,10,Newfoundland and Labrador,Not applicable,Not applicable,TOTAL,All Industries,"3,835","71,687,000","36,751",385
188,2020-03-15,10-rural,Newfoundland and Labrador - rural part,RURAL,Not applicable,TOTAL,All Industries,"1,150","17,397,000","8,735",130
283,2020-03-15,10-urban,Newfoundland and Labrador - urban part,URBAN,Not applicable,TOTAL,All Industries,"2,685","54,290,000","28,016",255
291,2020-03-15,1000000,Newfoundland and Labrador - rural part undeter...,RURAL,Not applicable,TOTAL,All Industries,5,"11,000",4,0
382,2020-03-15,10001,St. John's,URBAN,CMA,TOTAL,All Industries,"2,020","42,997,000","22,273",180


In [15]:
# Since "IndustryCode", "IndustryName" gives the same value, so we can remve them all.

data_CMA_CA_totalIndustry = data_CMA.drop(columns=["IndustryCode", "IndustryName"])
data_CMA_CA_totalIndustry.head(5)

,Start_date_of_CEWS_period,RegionCode,RegionName,RuralUrbanFlag,CMACAFlag,Number_business_locations,Subsidy_amount,Supported_employees,CEWS_rehire_count
98,2020-03-15,10,Newfoundland and Labrador,Not applicable,Not applicable,"3,835","71,687,000","36,751",385
188,2020-03-15,10-rural,Newfoundland and Labrador - rural part,RURAL,Not applicable,"1,150","17,397,000","8,735",130
283,2020-03-15,10-urban,Newfoundland and Labrador - urban part,URBAN,Not applicable,"2,685","54,290,000","28,016",255
291,2020-03-15,1000000,Newfoundland and Labrador - rural part undeter...,RURAL,Not applicable,5,"11,000",4,0
382,2020-03-15,10001,St. John's,URBAN,CMA,"2,020","42,997,000","22,273",180


In [16]:
# Add a new column called province
province_regionCode = data_province_totalIndustry[["RegionCode", "RegionName"]]
province_regionCode = province_regionCode[0:13]

data_CMA_CA_totalIndustry["province_regionCode"] = data_CMA_CA_totalIndustry["RegionCode"].str[0:2]  
data_CMA_CA_totalIndustry.head(5)

,Start_date_of_CEWS_period,RegionCode,RegionName,RuralUrbanFlag,CMACAFlag,Number_business_locations,Subsidy_amount,Supported_employees,CEWS_rehire_count,province_regionCode
98,2020-03-15,10,Newfoundland and Labrador,Not applicable,Not applicable,"3,835","71,687,000","36,751",385,10
188,2020-03-15,10-rural,Newfoundland and Labrador - rural part,RURAL,Not applicable,"1,150","17,397,000","8,735",130,10
283,2020-03-15,10-urban,Newfoundland and Labrador - urban part,URBAN,Not applicable,"2,685","54,290,000","28,016",255,10
291,2020-03-15,1000000,Newfoundland and Labrador - rural part undeter...,RURAL,Not applicable,5,"11,000",4,0,10
382,2020-03-15,10001,St. John's,URBAN,CMA,"2,020","42,997,000","22,273",180,10


In [17]:
province_regionCode.rename(columns={"RegionCode":"province_regionCode", "RegionName":"province_regionName"}, inplace=True)

In [18]:
# Perform a leter join:
data_CMA_CA_totalIndustry = pd.merge(data_CMA_CA_totalIndustry, province_regionCode, on = 'province_regionCode', how='left')
data_CMA_CA_totalIndustry.head()

,Start_date_of_CEWS_period,RegionCode,RegionName,RuralUrbanFlag,CMACAFlag,Number_business_locations,Subsidy_amount,Supported_employees,CEWS_rehire_count,province_regionCode,province_regionName
0,2020-03-15,10,Newfoundland and Labrador,Not applicable,Not applicable,"3,835","71,687,000","36,751",385,10,Newfoundland and Labrador
1,2020-03-15,10-rural,Newfoundland and Labrador - rural part,RURAL,Not applicable,"1,150","17,397,000","8,735",130,10,Newfoundland and Labrador
2,2020-03-15,10-urban,Newfoundland and Labrador - urban part,URBAN,Not applicable,"2,685","54,290,000","28,016",255,10,Newfoundland and Labrador
3,2020-03-15,1000000,Newfoundland and Labrador - rural part undeter...,RURAL,Not applicable,5,"11,000",4,0,10,Newfoundland and Labrador
4,2020-03-15,10001,St. John's,URBAN,CMA,"2,020","42,997,000","22,273",180,10,Newfoundland and Labrador


In [19]:
# remove any region codes that is or contains rural, urban.
data_CMA_CA_totalIndustry = data_CMA_CA_totalIndustry[data_CMA_CA_totalIndustry.RegionCode.str.get(2) != "-"]
data_CMA_CA_totalIndustry = data_CMA_CA_totalIndustry[(data_CMA_CA_totalIndustry.RegionCode != "rural")]
data_CMA_CA_totalIndustry = data_CMA_CA_totalIndustry[(data_CMA_CA_totalIndustry.RegionCode != "urban")]
# data_CMA_CA_totalIndustry = data_CMA_CA_totalIndustry[(data_CMA_CA_totalIndustry.RegionCode != "TOTAL")]

# Separate census division/census subdivision:

# Census divison:
data_CMA_CA_totalIndustry_CD = data_CMA_CA_totalIndustry[(data_CMA_CA_totalIndustry.RegionCode.str.len() == 5)]

# Census subdivison:
data_CMA_CA_totalIndustry_subCD = data_CMA_CA_totalIndustry[(data_CMA_CA_totalIndustry.RegionCode.str.len() == 7)]

# #we are looking at CMA/CA and Rural/Urban
# data_CMA_CA_totalIndustry = data_CMA_CA_totalIndustry[data_CMA_CA_totalIndustry.RegionCode.str.len() != 2]

In [20]:
data_CMA_CA_totalIndustry_CD.head()

,Start_date_of_CEWS_period,RegionCode,RegionName,RuralUrbanFlag,CMACAFlag,Number_business_locations,Subsidy_amount,Supported_employees,CEWS_rehire_count,province_regionCode,province_regionName
4,2020-03-15,10001,St. John's,URBAN,CMA,"2,020","42,997,000","22,273",180,10,Newfoundland and Labrador
5,2020-03-15,10005,Bay Roberts,URBAN,CA,70,"810,000",458,10,10,Newfoundland and Labrador
6,2020-03-15,10011,Gander,URBAN,CA,155,"3,532,000","1,648",25,10,Newfoundland and Labrador
51,2020-03-15,10015,Corner Brook,URBAN,CA,315,"5,274,000","2,549",20,10,Newfoundland and Labrador
295,2020-03-15,11105,Charlottetown,URBAN,CA,820,"13,496,000","7,625",70,11,Prince Edward Island


In [21]:
data_CMA_CA_totalIndustry_subCD.tail()

,Start_date_of_CEWS_period,RegionCode,RegionName,RuralUrbanFlag,CMACAFlag,Number_business_locations,Subsidy_amount,Supported_employees,CEWS_rehire_count,province_regionCode,province_regionName
26764,2020-09-27,6208047,Kugaaruk,RURAL,Not applicable,0,NaN,NaN,0,62,Nunavut
26765,2020-09-27,6208059,Kugluktuk,RURAL,Not applicable,0,"8,000",6,0,62,Nunavut
26766,2020-09-27,6208073,Cambridge Bay,RURAL,Not applicable,5,"124,000",117,0,62,Nunavut
26767,2020-09-27,6208081,Gjoa Haven,RURAL,Not applicable,0,NaN,NaN,0,62,Nunavut
26768,2020-09-27,6208087,Taloyoak,RURAL,Not applicable,0,NaN,NaN,0,62,Nunavut


In [22]:
data_CMA_CA_totalIndustry_CD.describe()

,Start_date_of_CEWS_period,RegionCode,RegionName,RuralUrbanFlag,CMACAFlag,Number_business_locations,Subsidy_amount,Supported_employees,CEWS_rehire_count,province_regionCode,province_regionName
count,944,944,944,944,944,944,932,933,944,944,936
unique,8,118,118,3,3,407,915,909,152,14,13
top,2020-03-15,24430,Ottawa - Gatineau (Ontario part),URBAN,CA,200,"1,300,000","3,105",10,35,Ontario
freq,118,8,8,912,624,17,2,2,80,224,224


In [23]:
data_CMA_CA_totalIndustry_subCD.describe()

,Start_date_of_CEWS_period,RegionCode,RegionName,RuralUrbanFlag,CMACAFlag,Number_business_locations,Subsidy_amount,Supported_employees,CEWS_rehire_count,province_regionCode,province_regionName
count,25508,25508,25508,25508,25508,25508,17069,17475,25508,25508,25508
unique,8,3439,3290,2,1,469,3802,3085,159,13,13
top,2020-05-10,3528052,Victoria,RURAL,Not applicable,0,0,12,0,24,Quebec
freq,3287,8,32,19131,25508,6863,117,165,18799,8302,8302


In [24]:
for i in ["RegionCode","CMACAFlag", "province_regionName"]:
    print(data_CMA_CA_totalIndustry_CD[i].unique())

['10001' '10005' '10011' '10015' '11105' '11110' '12205' '12210' '12215'
 '12220' '12225' '13305' '13310' '13320' '13328' '13329' '13330' '13335'
 '24330' '24403' '24404' '24405' '24406' '24408' '24410' '24411' '24412'
 '24421' '24430' '24433' '24440' '24442' '24444' '24447' '24450' '24452'
 '24454' '24456' '24462' '24480' '24505' '35501' '35505' '35507' '35509'
 '35512' '35515' '35521' '35522' '35529' '35532' '35535' '35537' '35539'
 '35541' '35543' '35550' '35555' '35556' '35557' '35559' '35562' '35566'
 '35568' '35571' '35575' '35580' '35590' '35595' '46602' '46603' '46610'
 '47705' '47710' '47715' '47720' '47725' '47735' '47745' '47750' '47840'
 '48805' '48806' '48810' '48825' '48831' '48835' '48860' '59905' '59907'
 '59913' '59915' '59918' '59920' '59925' '59930' '59932' '59933' '59934'
 '59935' '59937' '59938' '59939' '59940' '59943' '59944' '59945' '59950'
 '59952' '59955' '59965' '59970' '59977' '60000' '60990' '61000' '62000'
 'TOTAL']
['CMA' 'CA' 'Not applicable']
['Newfoundl

In [25]:
data_CMA_CA_totalIndustry_CD.to_csv("data_CMA_CA_totalIndustry_CD.csv")

In [26]:
data_CMA_CA_totalIndustry_subCD.to_csv("data_CMA_CA_totalIndustry_subCD.csv")

In [27]:
# Now, in order to complete the client's request, let's wrangle out data for each census subdivision areas with full industry observations 

In [43]:
# na_removed_data.IndustryCode.unique()

In [27]:
# List out all the census subdivisions, and remove total industry amount(avoid duplicate sum)
data_CMA_CA_explicit_Industry = na_removed_data[na_removed_data.RegionCode.str.len() == 7]
# data_CMA_CA_explicit_Industry = data_CMA_CA_explicit_Industry[na_removed_data.IndustryCode != "TOTAL"]

In [28]:
# Now add province label to the data set
data_CMA_CA_explicit_Industry["province_regionCode"] = data_CMA_CA_explicit_Industry["RegionCode"].str[0:2]
data_CMA_CA_explicit_Industry = pd.merge(data_CMA_CA_explicit_Industry, province_regionCode, on = 'province_regionCode', how='left')

<ipython-input-28-d04875b4b416>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_CMA_CA_explicit_Industry["province_regionCode"] = data_CMA_CA_explicit_Industry["RegionCode"].str[0:2]


In [29]:
data_CMA_CA_explicit_lv1_Industry = data_CMA_CA_explicit_Industry[data_CMA_CA_explicit_Industry.IndustryCode.str.len() != 3]
data_CMA_CA_explicit_lv2_Industry = data_CMA_CA_explicit_Industry[data_CMA_CA_explicit_Industry.IndustryCode.str.len() == 3]

In [30]:
data_CMA_CA_explicit_lv1_Industry.tail(6)

,Start_date_of_CEWS_period,RegionCode,RegionName,RuralUrbanFlag,CMACAFlag,IndustryCode,IndustryName,Number_business_locations,Subsidy_amount,Supported_employees,CEWS_rehire_count,province_regionCode,province_regionName
464000,2020-09-27,6208073,Cambridge Bay,RURAL,Not applicable,72,Accommodation and food services,0,NaN,NaN,0,62,Nunavut
464002,2020-09-27,6208073,Cambridge Bay,RURAL,Not applicable,TOTAL,All Industries,5,"124,000",117,0,62,Nunavut
464003,2020-09-27,6208081,Gjoa Haven,RURAL,Not applicable,48-49,Transportation and warehousing,0,NaN,NaN,0,62,Nunavut
464005,2020-09-27,6208081,Gjoa Haven,RURAL,Not applicable,TOTAL,All Industries,0,NaN,NaN,0,62,Nunavut
464006,2020-09-27,6208087,Taloyoak,RURAL,Not applicable,48-49,Transportation and warehousing,0,NaN,NaN,0,62,Nunavut
464008,2020-09-27,6208087,Taloyoak,RURAL,Not applicable,TOTAL,All Industries,0,NaN,NaN,0,62,Nunavut


In [31]:
data_CMA_CA_explicit_lv1_Industry.IndustryCode.unique()

array(['21', '23', '48-49', 'TOTAL', '51', '72', '31-33', '44-45', '53',
       '62', '52', '11', '56', '71', '99', '41', '54', '61', '55', '22'],
      dtype=object)

In [32]:
data_CMA_CA_explicit_lv2_Industry.IndustryCode.unique()

array(['213', '238', '484', '488', '492', '511', '722', '339', '447',
       '531', '236', '336', '623', '522', '445', '444', '311', '112',
       '237', '441', '524', '562', '713', '324', '413', '523', '316',
       '321', '446', '453', '114', '312', '624', '721', '337', '452',
       '416', '443', '448', '515', '517', '561', '621', '332', '412',
       '442', '327', '454', '414', '415', '418', '314', '419', '533',
       '417', '485', '487', '712', '111', '532', '323', '493', '519',
       '211', '212', '315', '334', '451', '481', '483', '512', '526',
       '711', '333', '113', '331', '322', '115', '325', '335', '326',
       '411', '486', '313', '518', '622', '491', '482'], dtype=object)

In [33]:
data_CMA_CA_explicit_lv1_Industry.Subsidy_amount.unique()

array([nan, '11,000', '0', ..., '11,485,000', '2,476,000', '3,902,000'],
      dtype=object)

In [34]:
# Convert to csv: 
data_CMA_CA_explicit_lv1_Industry.to_csv("data_CSD_explicit_lv1_Industry.csv")
data_CMA_CA_explicit_lv2_Industry.to_csv("data_CSD_explicit_lv2_Industry.csv")